START OF FILE

Start by setting up the notebook and the dataframe

In [1]:
#import everything that is necessary for this notebook
import pandas as pd
from sklearn.neural_network import MLPClassifier
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
#create dataframe from the dataset
df = pd.read_csv("music_genre.csv")

In this dataset, tempo uses ?'s for missing values, and duration has negative values within it. I wanted to see how many of each there are and how many of those are the same song. This would give me the knowledge of whether or not I should keep these columns.

In [2]:
#get the number of rows in the dataframe
size = len(df.index)
#initialize variables
badTempo = 0
badDuration = 0
badOverlap= 0

for x in range(size):
    #checks for bad tempos
    if df.at[x, "tempo"] == "?":
        badTempo += 1
        #checks if there is a bad tempo and a bad duration
        if df.at[x, 'duration_ms'] <= 0:
            badDuration += 1
            badOverlap += 1
    #checks if there is only a bad duration
    elif df.at[x, 'duration_ms'] <= 0:
        badDuration += 1
#prints out the results for comparison
print(badTempo)
print(badDuration)
print(badOverlap)

4980
4939
479


This shows that there is roughly 10,000 unusable values across these columns, with only 500 of those being on the same row. I made the decision that instead of deleting these rows, I would just delete the tempo and duration columns.

I also deleted the instance id, artist name, track name, and obtained date as these are all informational and will not help when determining the genre.

In [3]:
#deletes the columns
df = df.drop(columns=['instance_id', 'duration_ms', "artist_name", "track_name", "obtained_date", "tempo"])
#displays what is left
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
popularity,50000.0,44.220420,15.542008,0.000000,34.0000,45.000000,56.000000,99.000
acousticness,50000.0,0.306383,0.341340,0.000000,0.0200,0.144000,0.552000,0.996
danceability,50000.0,0.558241,0.178632,0.059600,0.4420,0.568000,0.687000,0.986
energy,50000.0,0.599755,0.264559,0.000792,0.4330,0.643000,0.815000,0.999
instrumentalness,50000.0,0.181601,0.325409,0.000000,0.0000,0.000158,0.155000,0.996
liveness,50000.0,0.193896,0.161637,0.009670,0.0969,0.126000,0.244000,1.000
loudness,50000.0,-9.133761,6.162990,-47.046000,-10.8600,-7.276500,-5.173000,3.744
speechiness,50000.0,0.093586,0.101373,0.022300,0.0361,0.048900,0.098525,0.942
valence,50000.0,0.456264,0.247119,0.000000,0.2570,0.448000,0.648000,0.992


The above table shows that loudness has both negative and positive values, since I eventually want all parameters to be between zero and one, I want to shift this. I also want to change the key, mode, and genre to numbers rather than objects so they will appear and can be normalized.

In [4]:

#store the inverse of the minimum value in the loudness column for later
minimum = df['loudness'].min() * -1

#loop through every single row to manipulate the data how I need it
for x in range(size):
    
    #Adds the inverse of the minumum loudness to every loudness, ensuring that all values are above or equal to zero
    df.at[x, 'loudness'] = df.at[x, 'loudness'] + minimum
    
    #------------------------------------
    #if the genre of the song is rock, it changes to 1, every other genre changes to 0, as rock is the target genre
    if df.at[x, "music_genre"] == "Rock":
        df.at[x, "music_genre"] = 1.0
    else: 
        df.at[x, "music_genre"] = 0.0
    
    #-------------------------------------
    #changes the key of the song to a number from 1-12 incrementally, A = 1, A# = 2, B = 3, and so on
    currentKey = df.at[x, "key"]
    if currentKey == "A":
        df.at[x, "key"] = 1.0

    elif currentKey == "A#":
        df.at[x, "key"] = 2.0

    elif currentKey == "B":
        df.at[x, "key"] = 3.0

    elif currentKey == "C":
        df.at[x, "key"] = 4.0

    elif currentKey == "C#":
        df.at[x, "key"] = 5.0

    elif currentKey == "D":
        df.at[x, "key"] = 6.0

    elif currentKey == "D#":
        df.at[x, "key"] = 7.0

    elif currentKey == "E":
        df.at[x, "key"] = 8.0

    elif currentKey == "F":
        df.at[x, "key"] = 9.0

    elif currentKey == "F#":
        df.at[x, "key"] = 10.0

    elif currentKey == "G":
        df.at[x, "key"] = 11.0

    else:
        df.at[x, "key"] = 12.0
    
    #------------------------------
    #if the mode of the song is major it becomes 1, if minor it becomes 0
    if df.at[x, "mode"] == "Major":
        df.at[x, "mode"] = 1.0

    else: 
        df.at[x, "mode"] = 0.0
    
print(df.dtypes)

popularity          float64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
valence             float64
music_genre          object
dtype: object


The table above shows that even after changing all of the data into numbers, the type of some columns are still objects.

In [5]:
#changes the data types from object to floats
df["music_genre"] = pd.to_numeric(df["music_genre"])
df["key"] = pd.to_numeric(df["key"])
df["mode"] = pd.to_numeric(df["mode"])
#confirms the types have changed
print(df.dtypes)

popularity          float64
acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                 float64
liveness            float64
loudness            float64
mode                float64
speechiness         float64
valence             float64
music_genre         float64
dtype: object


In [6]:
#show data after all changes
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
popularity,50000.0,44.220420,15.542008,0.000000,34.0000,45.000000,56.000000,99.000
acousticness,50000.0,0.306383,0.341340,0.000000,0.0200,0.144000,0.552000,0.996
danceability,50000.0,0.558241,0.178632,0.059600,0.4420,0.568000,0.687000,0.986
energy,50000.0,0.599755,0.264559,0.000792,0.4330,0.643000,0.815000,0.999
instrumentalness,50000.0,0.181601,0.325409,0.000000,0.0000,0.000158,0.155000,0.996
key,50005.0,6.354985,3.457513,1.000000,4.0000,6.000000,9.000000,12.000
liveness,50000.0,0.193896,0.161637,0.009670,0.0969,0.126000,0.244000,1.000
loudness,50000.0,37.912239,6.162990,0.000000,36.1860,39.769500,41.873000,50.790
mode,50005.0,0.641916,0.479442,0.000000,0.0000,1.000000,1.000000,1.000
speechiness,50000.0,0.093586,0.101373,0.022300,0.0361,0.048900,0.098525,0.942


The table above shows that all the data looks to now be in a useable form, however it still needs to be normalized.

In [7]:
#make it easier to perform functions on either only the target column or only the columns of the predictors
target = ['music_genre']
predictors = ['popularity', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'valence']

#saves the number of the predictors and the number of rows
outSize = len(predictors)
inSize = len(df.index)

#loops through each predictor
for x in range(outSize):
    currentPred = predictors[x]
    #gets maximum value for that column
    maximum = df[currentPred].max()
    for y in range(inSize):
        #divides each value by the maximum in it's column
        df.at[y, currentPred] = df.at[y, currentPred] / maximum
    

Now I wanted to remove all missing values from the DataFrame before splitting the data for training and testing

In [8]:
#Removes missing values
df = df.dropna()
#Use count column to verify
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
popularity,50000.0,0.446671,0.156990,0.000000,0.343434,0.454545,0.565657,1.0
acousticness,50000.0,0.307613,0.342711,0.000000,0.020080,0.144578,0.554217,1.0
danceability,50000.0,0.566167,0.181168,0.060446,0.448276,0.576065,0.696755,1.0
energy,50000.0,0.600355,0.264824,0.000793,0.433433,0.643644,0.815816,1.0
instrumentalness,50000.0,0.182330,0.326716,0.000000,0.000000,0.000159,0.155622,1.0
key,50000.0,0.529535,0.288102,0.083333,0.333333,0.500000,0.750000,1.0
liveness,50000.0,0.193896,0.161637,0.009670,0.096900,0.126000,0.244000,1.0
loudness,50000.0,0.746451,0.121343,0.000000,0.712463,0.783018,0.824434,1.0
mode,50000.0,0.641980,0.479423,0.000000,0.000000,1.000000,1.000000,1.0
speechiness,50000.0,0.099349,0.107615,0.023673,0.038323,0.051911,0.104591,1.0


The table above shows that now everything is between 0 and 1. Since it is now normalized, it is time to split the data into a training set and a testing set.

In [9]:
#changes the dataframe into two numpy arrays
x = df[predictors].values
y = df[target].values
#Change y from a column-vector to a 1d array
y = np.ravel(y)
#split x and y arrays into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state=40)

Make Classifier model

In [10]:
#creates the model
mlp = MLPClassifier(hidden_layer_sizes=(11,11,11), activation='relu', solver='adam', max_iter=500)
#fits model to the training data
mlp.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(11, 11, 11), max_iter=500)

In [11]:
#makes predictions on the data used for training
predict_train = mlp.predict(x_train)
#makes predictions on new testing data
predict_test = mlp.predict(x_test)

Now time to evaluate

In [12]:
from sklearn.metrics import classification_report,confusion_matrix
#shows results of predictions of training data
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train, predict_train))

[[30647   865]
 [ 2156  1332]]
              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95     31512
         1.0       0.61      0.38      0.47      3488

    accuracy                           0.91     35000
   macro avg       0.77      0.68      0.71     35000
weighted avg       0.90      0.91      0.90     35000



Now evaluate with the test data

In [13]:
#shows results of predictions of training data
print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test, predict_test))

[[13115   373]
 [  920   592]]
              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95     13488
         1.0       0.61      0.39      0.48      1512

    accuracy                           0.91     15000
   macro avg       0.77      0.68      0.72     15000
weighted avg       0.90      0.91      0.91     15000

